In [30]:
import os
import numpy as np
from nilearn import plotting, image
import pandas as pd

In [39]:
#loading the data
def load_txt(n=100, file_path=None):
    '''
    loads txt data of all subjects

    params:
    n (int): number of brain regions options: 15, 50, 100; 100 by default
    subset (tuple): gets a subset of subjects; (start, stop, step) ... eventually 
    file_path: your own filepath

    return:
    all data (np.array): 
    '''
    if file_path:
        file_path = file_path
    else:
        file_path = f"/home/anmarkova/teams/a05/group_2/HCP_PTN1200/node_timeseries/3T_HCP1200_MSMAll_d{n}_ts2"
    sub_list = os.listdir(file_path)
    make_path = lambda x: file_path + "/" + x
    paths = [make_path(i) for i in sub_list]
    sub_data = [np.loadtxt(path) for path in paths]

    print('Subject Data Loaded')

    return np.array(sub_data)

In [40]:
#loading atlas parcelations
def load_atlas(n=100, file_path=None):
    '''
    loads txt data of all subjects

    params:
    n (int): number of brain regions options: 15, 50, 100; 100 by default
    file_path: your own filepath

    return:
    all data (np.array): 
    '''
    if not file_path:
        file_path = f"/home/anmarkova/teams/a05/group_2/HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d{n}.ica/melodic_IC_sum.nii.gz"
    atlas = image.load_img(f"/home/anmarkova/teams/a05/group_2/HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d{n}.ica/melodic_IC_sum.nii.gz")
    atlas = image.threshold_img(atlas, "99.5%") 

    print("atlas has shape", ["x", "y", "z", "region"], "=", atlas.shape)
    return atlas

In [41]:
#load unrestricted data
def load_unrestricted(file_path=None):

    if not file_path:
        file_path = "/home/anmarkova/teams/a05/group_2/unrestricted_data.csv"
    data = pd.read_csv(file_path)
    print('unrestricted_data loaded')
    return data

In [42]:
#load restricted data
def load_restricted(file_path=None):

    if not file_path:
        file_path = "/home/anmarkova/teams/a05/group_2/RESTRICTED_BEHAVIORAL_DATA.csv"
    data = pd.read_csv(file_path)
    print('restricted_data loaded')
    return data